In [1]:
# TODO update interest rate logic on offering 

# Electraseed Energy Financial Model - Version 1: Anchor

## Purpose
The energy financial model version 1 evaluates the financial performance of an aggregated pool of loans to clean energy developers, and the resulting outcomes for the investor, the developers, the Electraseed Fund, and any 3rd party actors.

## Structure
The basic structure is comprised of the following components:  

**Inputs**  
* Library Import  
* Input Definition  

**Actors**  
* Actor Class Definition  
* Actor Creation  

**Loan Creations**  
* Loan Menu Creation  
* Portfolio Optimizer  
* Loan Matching  

**Investment Projections**  
* Investment Math  
* Risk Projections  

**Outputs**  
* Output Aggregation  
* Output Dashboard

## Energy Model

### Library Import
Import libraries requried for energy financial model

In [2]:
import numpy as np
import pandas as pd
import scipy.optimize as sco
import random
import math

### Input Definition
Inputs are entered here as described above

In [3]:
# Invididual Energy Asset Information
# TODO2 get names of individual projects
# TODO2 get actual numbers
dea_names = ['Polarstern', 'MaxSolar', 'Solartainer', 'SOLShare'] # name of dea
dea_locations = ['Germany', 'Germany', 'Mali', 'Bangladesh'] # location of dea
dea_classifications = ['developed', 'developed', 'developing', 'developing'] # classification of economy
dea_loansizerequests = [2000000,1000000,700000,500000] # euros loan requested
dea_rateappetites = [0.04, 0.07, 0.10, 0.16] # % annual interest appetite
dea_defaultrates = [0.04, 0.05, 0.15, 0.20] # % annual risk of default
dea_paymentfreqs = [1, 1, 1, 12] # months between interest payments
dea_loanperiods = [15, 15, 10, 15] # years of loan period

# Investor Information
investor_names = ['altruistic', 'impact1', 'impact2', 'impact3', 'monopolyman'] # name of investor
investor_investment_requests = [1000000, 1000000, 1000000, 1000000, 1000000] # euros investment
investor_ratefloors = [0.00, 0.05, 0.04, 0.03, 0.07] # % annual interest minimum
investor_riskappetites = [0.08, 0.04, 0.06, 0.06, 0.02] # % annual risk of default
investor_riskfreerates = [0.01, 0.01, 0.01, 0.01, 0.01]

# Underwriter Information
uw_names = ['1', '2', '3', '4']
uw_locations = ['Germany', 'Mali', 'Bangladesh', 'Bangladesh']
uw_maxloans = [5000000, 600000, 200000, 400000] # euros max underwrite
uw_rateinvs = [0.03, 0.09, 0.11, 0.11] # % annual interest rate OFFER
uw_ratedeas = [0.035, 0.10, 0.12, 0.12] # % annual interest minimum LOAN OUT
uw_riskappetites = [0.03, 0.15, 0.15, 0.15] # % annual risk of default appetite

# spv Information
spv_names = ['Electraseed']
spv_interestratios = [0.15]

### Actor Class Definition
Define digital energy asset, investor, and other actor classes

In [4]:
# Digital Energy Asset (DEA) class
class dea_: 
    def __init__(self, name, location, classification, loansizerequest, rateappetite, defaultrate, paymentfreq, loanperiod):
        # attributes
        self.name = name
        self.location = location
        self.classification = classification
        self.loansizerequest = loansizerequest
        self.rateappetite = rateappetite
        self.defaultrate = defaultrate
        self.paymentfreq = paymentfreq
        self.loanperiod = loanperiod
        # loan tracking
        self.activeloanamount = 0
        self.indexnum = None
        # TODO3 add energy data
        
# Investor class
class investor_:
    def __init__(self, name, investment_request, ratefloor, riskappetite, riskfreerate):
        self.name = name
        self.investment_request = investment_request
        self.ratefloor = ratefloor
        self.riskappetite = riskappetite
        self.riskfreerate = riskfreerate
        # TODO2 add optimization metric
        
# Underwriter class
class underwriter_:
    def __init__(self, name, location, maxloan, rateinv, ratedea, riskappetite):
        self.name = name
        self.location = location
        self.maxloan = maxloan
        self.rateinv = rateinv
        self.ratedea = ratedea
        self.riskappetite = riskappetite
        # loan tracking
        self.activedealoanamount = 0
        self.activespvloanamount = 0
        
# Elecraseed class
class spv_:
    def __init__(self, name, interestratio):
        self.name = name
        self.interestratio = interestratio

### Actor Creation
Create the actual entities involved in our ecosystem, filling out their appropriate attributes.  
Each of the entities are placed in a list of all existing entities of that class

In [5]:
# Digital Energy Assets
deas = []
for i in range(len(dea_names)):
    deas.append(dea_(dea_names[i], dea_locations[i], dea_classifications[i], dea_loansizerequests[i], dea_rateappetites[i], dea_defaultrates[i], dea_paymentfreqs[i], dea_loanperiods[i]))
    deas[i].indexnum = i # set index num for later reference
    
# Investors
investors = []
for i in range(len(investor_names)):
    investors.append(investor_(investor_names[i], investor_investment_requests[i], investor_ratefloors[i], investor_riskappetites[i], investor_riskfreerates[i]))

# Underwriters
underwriters = []
for i in range(len(uw_locations)):
    underwriters.append(underwriter_(uw_names[i], uw_locations[i], uw_maxloans[i], uw_rateinvs[i], uw_ratedeas[i], uw_riskappetites[i]))

# spvs
spvs = [spv_(spv_names, spv_interestratios)]


### Loan Menu Creation
Create the 'menu' of loans available to investors  
#### Create the Loan Class

In [6]:
# Loan class
class subloan_:
    def __init__(self, investor, rateinv_precut, rateinv_postcut, amount):
        self.investor = investor
        self.rateinv_precut = rateinv_precut
        self.rateinv_postcut = rateinv_postcut
        self.amount = amount
        # payment tracker
        # inv precut
        self.payment_inv_precut = None
        self.payment_interest_inv_precut = []
        self.payment_principal_inv_precut = []
        # inv postcut
        self.payment_inv_postcut = None
        self.payment_interest_inv_postcut = []
        self.payment_principal_inv_postcut = []
        
        
class loan_:
    def __init__(self, dea, underwriter, amount, ratedea, paymentfreq, loanperiod, defaultrate):
        self.dea = dea
        self.underwriter = underwriter
        self.amount = amount
        self.ratedea = ratedea
        self.paymentfreq = paymentfreq
        self.loanperiod = loanperiod
        self.defaultrate = defaultrate
        # loan tracking
        self.amountfinanced = 0
        self.subloans = []
        # payment tracker
        # dea
        self.payment_dea = None
        self.payment_interest_dea = []
        self.payment_principal_dea = []

#### Loan Requests Fracturing and Loan Menu Creation
Fracture the loan requests based on the underwriter's loan offerings.  
Add each of these to a loan 'menu' to pair with investors

In [7]:
# Underwriter - DEA loan matching

# match deas to underwriters if loan is possible
loan_menu = []
total_loan_amount_requested = 0
total_amount_in_loan_options = 0
for dea in deas:
    total_loan_amount_requested += dea.loansizerequest
    if dea.activeloanamount < dea.loansizerequest: # check if loan needed
        for underwriter in underwriters:
            # TODO2 add sorting script for loan rates
            if underwriter.location == dea.location: # check if eligible to loan
                if underwriter.ratedea <= dea.rateappetite: # check rate eligible
                    ratedea = underwriter.ratedea # rate is underwriters offer to dea
                    rateinv = underwriter.rateinv
                    amount = min(underwriter.maxloan-underwriter.activedealoanamount, dea.loansizerequest-dea.activeloanamount) # amount is what dea asks for or max underwriter offer
                    loan = loan_(dea, underwriter, amount, ratedea, dea.paymentfreq, dea.loanperiod, dea.defaultrate)
                    underwriter.activedealoanamount += amount # track active loans
                    dea.activeloanamount += amount # track active loan amount
                    total_amount_in_loan_options += amount #
                    loan_menu.append(loan) # track active loans in loan menu

print("Total loan amount requested:", total_loan_amount_requested)
print("Number of loan options created:", len(loan_menu))
print("Total loan amount fundable:", total_amount_in_loan_options)
            
        
    



Total loan amount requested: 4200000
Number of loan options created: 5
Total loan amount fundable: 4100000


#### Covariance Matrix Definition
Create the covariance matrix for risk of the projected loans

In [8]:
# Create covariance matrix
def covariance_matrix_creator(loans):
    location_same_cov = 0.15 # cov impact if in same location
    location_dif_cov = 0. # cov impact if in different location
    classification_same_cov = 0.15 # cov impact if same classification
    classification_dif_cov = -0.1 # cov impact if different classification
    same_borrower_dif_cov = 0.9
    cov_matrix = np.zeros((len(loans), len(loans))) # initialize covariance matrix
    for i in range(len(loans)):
        for j in range(i, len(loans)):
            if i == j: 
                cov_matrix[i][j] = 1 # set diagonal to 1
            else:
                if loans[i].dea.name == loans[j].dea.name: # check if same name
                    cov_matrix[i][j] = same_borrower_dif_cov
                    cov_matrix[j][i] = same_borrower_dif_cov
                else:
                    if loans[i].dea.location == loans[j].dea.location: # check for location cov
                        cov_matrix[i][j] += location_same_cov
                        cov_matrix[j][i] = cov_matrix[i][j]
                    else:
                        cov_matrix[i][j] += location_dif_cov
                        cov_matrix[j][i] = cov_matrix[i][j]
                    if loans[i].dea.classification == loans[j].dea.classification: # check for economy classification cov
                        cov_matrix[i][j] += classification_same_cov
                        cov_matrix[j][i] = cov_matrix[i][j]
                    else:
                        cov_matrix[i][j] += classification_dif_cov
                        cov_matrix[j][i] = cov_matrix[i][j]
    return cov_matrix

cov_matrix = covariance_matrix_creator(loan_menu)

### Portfolio Optimizer Definition
Before we can match loans to investors, we need to create 'optimal' portfolio math based on investor-specific inputs

In [9]:
# define function to calculate portfolio performance
def portfolio_performance(weights, returns, stds, cov_matrix):
    p_ret = np.sum(weights * np.asarray(returns)) # calculate return
    # calculate std using 'modern portfolio theory', calculations available on wiki
    p_std = 0 
    for i in range(len(weights)): # summation of each weights^2 and std^2
        p_std += (weights[i]**2)*(stds[i])**2
    for i in range(len(weights)): # summation of covariance calculation
        for j in range(len(weights)):
            if i != j:
                p_std += weights[i]*weights[j]*stds[i]*stds[j]*cov_matrix[i][j]
    p_std = np.sqrt(p_std) # sqrt for variance --> std
    # TODO check risk calculation for type of loan
    return p_std, p_ret

# define function to get sharpe ratio for optimization measure
def neg_sharpe_ratio(weights, returns, stds, cov_matrix, risk_free_rate):
    p_std, p_ret = portfolio_performance(weights, returns, stds, cov_matrix) # get std and return
    return -(p_ret - risk_free_rate) / p_std # calculate sharpe ratio

#TODO define function to get social / environmental optimization

# define function to find optimal portfolio weights
def portfolio_optimizer(loan_menu, cov_matrix, investor, return_rates, stds, amounts):
    # TODO2 optimize for impact
    # TODO2 instead return efficient frontier, maybe then can optimize further for impact?
    # set investor investment amount to min amount remaining against requested amount
    investor.investment = min(investor.investment_request, np.sum(amounts))
    num_assets = len(loan_menu)
    args = (return_rates, stds, cov_matrix, investor.riskfreerate)
    # set helper functions for constraints
    def portfolio_risk(weights):
        return portfolio_performance(weights, return_rates, stds, cov_matrix)[0]
    def portfolio_return(weights):
        return portfolio_performance(weights, return_rates, stds, cov_matrix)[1]
    # define constraints
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}, # weights sum to 1
                   {'type': 'ineq', 'fun': lambda x: amounts - x * investor.investment}, # investor investment <= requested loan amount
                   {'type': 'ineq', 'fun': lambda x: investor.riskappetite - portfolio_risk(x)}, # portfolio risk <= investor risk appetite
                   {'type': 'ineq', 'fun': lambda x: portfolio_return(x) - investor.ratefloor}) # portfolio return >= investor rate floor
                   
    bound = (0.0, 1.0) # set bounds for the portfolio weights
    bounds = tuple(bound for asset in range(num_assets))
    # run actual optimization on negative sharpe ratio (will move to impact ratio)
    result = sco.minimize(neg_sharpe_ratio, num_assets*[1./num_assets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def optimize_portfolio(loan_menu, cov_matrix, investor, return_rates, return_rates_investor, stds, amounts):
    opt = portfolio_optimizer(loan_menu, cov_matrix, investor, return_rates_investor, stds, amounts) # get optimal portfolio base on inputs
    weights = opt.x # extract output weights
    # print some relevant outputs
    print('--------')
    print('investor:', investor.name)
    print('investment:',np.round(investor.investment),"of",investor.investment_request,"requested")
    print('risk tol:', investor.riskappetite, 'rate floor:', investor.ratefloor, 'risk free:', investor.riskfreerate)
    if opt.success:
        print('optimal portfolio found:')
    else:
        print('not all constraints met. Best we could do:')
    p_std, p_ret = portfolio_performance(weights, return_rates, stds, cov_matrix) # get optimal portfolio performance
    print('\ttot_risk:',np.round(p_std*100,2),'%,', 'tot_return:',np.round(p_ret*100,2),'%,', 'tot_sharpe:', np.round((p_ret-investor.riskfreerate)/p_std,2))
    p_std, p_ret = portfolio_performance(weights, return_rates_investor, stds, cov_matrix) # get optimal portfolio performance
    print('\tinv_risk:',np.round(p_std*100,2),'%,', 'inv_return:',np.round(p_ret*100,2),'%,', 'inv_sharpe:', np.round((p_ret-investor.riskfreerate)/p_std,2))
    print('\tportfolio weights', np.round(weights,2))
    print('\tportfolio investments', np.round(weights*investor.investment))
    print('\tloan requested', np.round(amounts))
    return opt.success, weights

### SPV Business Case
Addd tool to adjust rate that investor sees to allow for SPV revenue from interest payments. 

In [10]:
# calculate new interest rate for investor based on spv cut
def investor_interest_after_spv_cut(rates, spv):
    investor_rates = np.asarray(rates) * (1.-spv.interestratio[0]) # cut interest rates by ratio
    return investor_rates

### Loan Matching
Tinder for loans

In [11]:
# run portfolio optimization for each investor

# next iterate through each investor to find optimal portfolio
for investor in investors:
    # first get return rates, stds, and loan amounts requested from the loan options
    return_rates = []
    stds = []
    amounts = []
    for loan_option in loan_menu:
        amount = loan_option.amount - loan_option.amountfinanced # get loan amount requested
        amounts.append(amount)
        # if there is no more money requested, make that loan give 0% return to trick model
        if amount > 1:
            return_rates.append(loan_option.underwriter.rateinv) # get return rate for investors
            stds.append(loan_option.defaultrate) # get risk rate
        else:
            return_rates.append(0.)
            stds.append(1.)
    return_rates_investor = investor_interest_after_spv_cut(return_rates, spvs[0])
    # optimize the portfolio under the constraints
    success, weights = optimize_portfolio(loan_menu, cov_matrix, investor, return_rates, return_rates_investor, stds, amounts)
    # update investor tracker
    investor.bestportfolio = weights
    investor.optimalportfoliofound = success
    # update loan menu with amount invested
    i = 0
    for loan_option in loan_menu:
        amount = weights[i] * investor.investment
        loan_option.amountfinanced += amount
        rateinv_postcut = investor_interest_after_spv_cut(loan_option.underwriter.rateinv, spvs[0])
        subloan = subloan_(investor, loan_option.underwriter.rateinv, rateinv_postcut, amount)
        if subloan.amount >= 1:
            loan_option.subloans.append(subloan)
        i += 1
    # TODO adjust loan menu based on investment
    

--------
investor: altruistic
investment: 1000000 of 1000000 requested
risk tol: 0.08 rate floor: 0.0 risk free: 0.01
optimal portfolio found:
	tot_risk: 3.83 %, tot_return: 4.72 %, tot_sharpe: 0.97
	inv_risk: 3.83 %, inv_return: 4.01 %, inv_sharpe: 0.79
	portfolio weights [0.48 0.27 0.14 0.05 0.05]
	portfolio investments [475807. 273155. 144083.  53477.  53477.]
	loan requested [2000000 1000000  600000  200000  300000]
--------
investor: impact1
investment: 1000000 of 1000000 requested
risk tol: 0.04 rate floor: 0.05 risk free: 0.01
not all constraints met. Best we could do:
	tot_risk: 5.46 %, tot_return: 5.88 %, tot_sharpe: 0.89
	inv_risk: 5.46 %, inv_return: 5.0 %, inv_sharpe: 0.73
	portfolio weights [0.36 0.23 0.24 0.09 0.09]
	portfolio investments [355661. 225116. 235718.  91753.  91753.]
	loan requested [1524193.  726845.  455917.  146523.  246523.]
--------
investor: impact2
investment: 1000000 of 1000000 requested
risk tol: 0.06 rate floor: 0.04 risk free: 0.01
optimal portfoli

### Investment Math
Calculate payments and amortization schedule, to define which parties are getting which amounts at which time.
#### Loan Status  
Print existing loan information for reference

In [12]:
# print useful information about loans and subloans
for loan in loan_menu:
    print("dea:",loan.dea.name,", underwriter:",loan.underwriter.name)
    print("financed:",np.round(loan.amountfinanced),"of",loan.amount,"with",len(loan.subloans),"subloans:")
    for subloan in loan.subloans:
        print("\t","investor:",subloan.investor.name,"amount:",np.round(subloan.amount),"rate_precut:",subloan.rateinv_precut,"rate_postcut:",subloan.rateinv_postcut)
    print("-----------")
        

dea: Polarstern , underwriter: 1
financed: 2000000.0 of 2000000 with 5 subloans:
	 investor: altruistic amount: 475807.0 rate_precut: 0.03 rate_postcut: 0.0255
	 investor: impact1 amount: 355661.0 rate_precut: 0.03 rate_postcut: 0.0255
	 investor: impact2 amount: 475876.0 rate_precut: 0.03 rate_postcut: 0.0255
	 investor: impact3 amount: 592657.0 rate_precut: 0.03 rate_postcut: 0.0255
	 investor: monopolyman amount: 100000.0 rate_precut: 0.03 rate_postcut: 0.0255
-----------
dea: MaxSolar , underwriter: 1
financed: 1000000.0 of 1000000 with 4 subloans:
	 investor: altruistic amount: 273155.0 rate_precut: 0.03 rate_postcut: 0.0255
	 investor: impact1 amount: 225116.0 rate_precut: 0.03 rate_postcut: 0.0255
	 investor: impact2 amount: 273133.0 rate_precut: 0.03 rate_postcut: 0.0255
	 investor: impact3 amount: 228596.0 rate_precut: 0.03 rate_postcut: 0.0255
-----------
dea: Solartainer , underwriter: 2
financed: 600000.0 of 600000 with 4 subloans:
	 investor: altruistic amount: 144083.0 ra

#### Loan Payment Calculations
Calculate payments by deas and underwriters


In [13]:
def get_payments(num_periods, rate, amount):
    principals = []
    interests = []
    payments = []
    for period in range(int(num_periods)):
        principal = -np.ppmt(rate, period, num_periods, amount) # principal payment calc
        principals.append(principal)
        interest = -np.ipmt(rate, period, num_periods, amount) # interest payment calc
        interests.append(interest)
        payment = principal + interest
        payments.append(payment)
    return principals, interests, payments

for loan in loan_menu:
    num_periods = int(loan.loanperiod * (12./loan.paymentfreq)) # get number of periods
    ratedea = loan.ratedea / (12./loan.paymentfreq) # get dea rate
    amount = loan.amountfinanced # get dea amount to pay
    # get payments for dea
    principal_dea, interest_dea, payment_dea = get_payments(num_periods, ratedea, amount)
    loan.payment_dea = payment_dea
    loan.payment_interest_dea = interest_dea
    loan.payment_principal_dea = principal_dea
    print("dea m payment:",np.round(payment_dea[0]),"amount:",np.round(loan.amount),"rate:",loan.ratedea)
    print("Subloans:")
    # get payments for subloans from underwriter to investors    
    for subloan in loan.subloans:
        amount = subloan.amount / (1.2/loan.paymentfreq) # get subloan amount
        rateinv_precut = subloan.rateinv_precut / (12./loan.paymentfreq) # get precut investor rate
        rateinv_postcut = subloan.rateinv_postcut / (12./loan.paymentfreq) # get postcut investor rate
        # calculate interest and principle
        # for investor precut
        principal_inv_precut, interest_inv_precut, payment_inv_precut = get_payments(num_periods, rateinv_precut, amount)
        subloan.payment_inv_precut = payment_inv_precut
        subloan.interest_inv_precut = interest_inv_precut
        subloan.principal_inv_precut = principal_inv_precut
        # for investor postcut
        principal_inv_postcut, interest_inv_postcut, payment_inv_postcut = get_payments(num_periods, rateinv_postcut, amount)
        subloan.payment_inv_postcut = payment_inv_postcut
        subloan.interest_inv_postcut = interest_inv_postcut
        subloan.principal_inv_postcut = principal_inv_postcut
        print("\t","amount:",np.round(subloan.amount),"rate_pre:",subloan.rateinv_precut,"rate_post:",subloan.rateinv_postcut)
        print("\t","underwriter m payment:",np.round(payment_inv_precut[0]))
        print("\t","investor receives:",np.round(payment_inv_postcut[0]))
        print("\t","Electraseed receives:",np.round(payment_inv_precut[0]-payment_inv_postcut[0]))
        print("\t","-----------------------")

dea m payment: 14298.0 amount: 2000000 rate: 0.035
Subloans:
	 amount: 475807.0 rate_pre: 0.03 rate_post: 0.0255
	 underwriter m payment: 2738.0
	 investor receives: 2653.0
	 Electraseed receives: 85.0
	 -----------------------
	 amount: 355661.0 rate_pre: 0.03 rate_post: 0.0255
	 underwriter m payment: 2047.0
	 investor receives: 1983.0
	 Electraseed receives: 64.0
	 -----------------------
	 amount: 475876.0 rate_pre: 0.03 rate_post: 0.0255
	 underwriter m payment: 2739.0
	 investor receives: 2654.0
	 Electraseed receives: 85.0
	 -----------------------
	 amount: 592657.0 rate_pre: 0.03 rate_post: 0.0255
	 underwriter m payment: 3411.0
	 investor receives: 3305.0
	 Electraseed receives: 106.0
	 -----------------------
	 amount: 100000.0 rate_pre: 0.03 rate_post: 0.0255
	 underwriter m payment: 575.0
	 investor receives: 558.0
	 Electraseed receives: 18.0
	 -----------------------
dea m payment: 7149.0 amount: 1000000 rate: 0.035
Subloans:
	 amount: 273155.0 rate_pre: 0.03 rate_post: 

### Risk Projections
Get confidence intervals on the return of the investment math based on loan default projections.  
This could be a good test case / playground for some simple cadCAD implementation

### Output Aggregation
Gather relevant outputs

### The Closest Thing to a Dashboard We Should Do in Jupyter Notebooks
Visualize relevant info!